### Notebook implementation of the 'gen_feature_matched_face_pairs7_magneto.py" That Liang gifted me
#### This produces the best match 1-1 match for a given familiar face. The algorithm has to be run a few times with ~1k iterations per run
#### Open question: How to do QA on the results given?
#### March 2023

#### Make sure paths to features and AAM model are set correctly

In [1]:
import socket
print(socket.gethostname())

DWA644201


In [2]:
# -*- coding: utf-8 -*-
# gen face similar to target face (fixed small distance in face space) 
# and low level feature matched

# Distribution difference measure: KL divergence
# Match outer contour also

#%% 

import numpy as np
# import concurrent.futures
# import matplotlib.pyplot as plt

from scipy import optimize
from scipy import io
from skimage.color import rgb2hsv, rgb2gray
import pickle


#%% input - dunno what this is doing
# import sys
# ModelPath = 'C:/Users/varunwadia/Documents/PYTHON/FaceFamiliarity/face_model_human/Model'
# sys.path.append(ModelPath)
# if len(sys.argv)>1:
#     iface = int(sys.argv[1])
# else:
#     iface = 0

#%% import model data
import os
import socket 
if socket.gethostname() == 'DWA644201':
    model_data = 'C:/Users/wadiav/Documents/PYTHON/FaceFamiliarity/face_model_human/Model_Data/Human_Face_Model_Data.mat'
elif socket.gethostname() == 'DESKTOP-LJHLIED':
    model_data = 'C:/Users/varunwadia/Documents/PYTHON/FaceFamiliarity/face_model_human/Model_Data/Human_Face_Model_Data.mat'
else:
    model_data = '/Users/varunwadia/Documents/PYTHON/FaceFamiliarity/face_model_human/Model_Data/Human_Face_Model_Data.mat'



In [3]:
#%% optimization options

op_method = 'BFGS'
options_optimize = {'disp': True, 'maxiter': 10, 'gtol': 1e-03, 'eps': 1e-02}

# op_method = 'Nelder-Mead'
# options_optimize = {'disp': True, 'maxiter': 1000}


#%% initialize face generator
from Model.AAM_Model import AAM_Model
# from Model import AAM_Model

model = AAM_Model(model_data)
output_res = [360, 256]
options_face_gen = {'normalized': False, 'ndim_shape': 20}

#%% familiar faces params (shape-appearance feature)
p_fam = io.loadmat('params_fam_100d.mat')
p_fam = p_fam['p_fam']
n_fam = p_fam.shape[0]
n_dim = p_fam.shape[1]

#%%
rng = np.random.default_rng()


#%% get image features
bin_hue = np.arange(0,1.01,0.01)
bin_low = np.arange(-10,10,0.5)


In [4]:
def get_image_features_distribution(im):
    
    im_gray = rgb2gray(im.squeeze())
    
    lum = im_gray.mean()
    contrast = np.square(im_gray-128).mean()
    
    im_hsv = rgb2hsv(im.squeeze())
    hue = im_hsv[:,:,0].flatten()
    hue_pdf = np.histogram(hue, bin_hue)[0]/len(hue)

    return lum, contrast, hue_pdf


#%% optimize



def find_low_level_matched_face(p_fam1, deviation):
    #familiar face low level features
    
    im_fam, landmarks_fam = model.gen_image_param(p_fam1.reshape((1,-1)), output_res, options_face_gen)
    rim_fam = landmarks_fam[model.data.mark_group.rim-1,:] # 0 indexing for python
    
    lum_fam, contrast_fam, hue_fam_pdf = get_image_features_distribution(im_fam)
    hue_fam_entropy = - np.dot(hue_fam_pdf, np.log2(hue_fam_pdf+1e-6))
    
    def cost_fun(dp):
        
        ## constraint in distance of deviation vector
        dp = dp/np.sqrt(np.inner(dp, dp)/len(dp))*deviation
        
        p = p_fam1 + dp
                
        im, landmarks = model.gen_image_param(p.reshape((1,-1)), output_res, options_face_gen)
        rim = landmarks[model.data.mark_group.rim-1,:]
        
        lum, contrast, hue_pdf = get_image_features_distribution(im)
        
        # contrast
        cost1 = abs(contrast_fam - contrast)/100
        
        # luminance
        cost2 = abs(lum_fam - lum)/10
        
        # distribution hue
        cost3 = (-np.dot(hue_fam_pdf, np.log2(hue_pdf+1e-6)) - hue_fam_entropy)*10
        
        # rim
        cost4 = np.mean((rim_fam - rim)**2)/200
        
        # total cost
        cost = cost1 + cost2 + cost3 + cost4
        
        # print(f'cost:{cost:.4f},contr:{cost1:.4f},lum:{cost2:.4f},hue:{cost3:.4f},rim:{cost4:.4f}')
        # print(time.time())
        
        return cost
    
    dp0 = rng.normal(scale = deviation, size = p_fam1.shape)    
    
    print ('start optimization...')
    res = optimize.minimize(cost_fun, dp0, method = op_method, options = options_optimize)
    
    return res


In [5]:
#%%
import time
t_start = time.time()


# takes ~6min for each iteration of the for loop
# Should I do this for each 
deviation = 1.5
iface = 0
# results = []
for i in range(5): # gradient descent iterations?
    
    print (f'face{iface}:')
    p_fam1 = p_fam[iface,:]
    r = find_low_level_matched_face(p_fam1, deviation)
    # results.append(r)
    
    with open(f'./results/result{iface:02d}_{i:02d}.pickle', 'wb') as f:
        pickle.dump(r, f)


t_end = time.time()
t_diff = t_end - t_start
print(f'elapsed time {t_diff} seconds')

face0:
start optimization...
         Current function value: 4.921699
         Iterations: 9
         Function evaluations: 4755
         Gradient evaluations: 47
face0:
start optimization...
         Current function value: 4.118010
         Iterations: 10
         Function evaluations: 1515
         Gradient evaluations: 15
face0:
start optimization...
         Current function value: 4.038640
         Iterations: 10
         Function evaluations: 1616
         Gradient evaluations: 16
face0:
start optimization...
         Current function value: 3.088211
         Iterations: 10
         Function evaluations: 1313
         Gradient evaluations: 13
face0:
start optimization...
         Current function value: 1.566708
         Iterations: 10
         Function evaluations: 1414
         Gradient evaluations: 14
elapsed time 2799.8648324012756 seconds


im_fam, landmarks_fam = model.gen_image_param(p_fam1.reshape((1,-1)), output_res, options_face_gen)
rim_fam = landmarks_fam[model.data.mark_group.rim-1,:]

 with open(f'./results/result{iface:02d}_{i:02d}.pickle', 'wb') as f:
        pickle.dump(r, f)

%matplotlib inline 
from matplotlib import pyplot as plt
plt.imshow(im_fam)
plt.show()

i

In [8]:
iface = 0
i = 0
total_cost = np.zeros(shape=(1, 5))
for i in range(5):
    with open(f'./results/result{iface:02d}_{i:02d}.pickle', 'rb') as f:
        dest_object = pickle.load(f)
        total_cost[0,i] =  dest_object.fun

In [9]:
total_cost

array([[4.92169865, 4.11800969, 4.03863992, 3.08821099, 1.56670814]])

len(dest_object.final_simplex[1])
